## Loading Data

In [63]:
import pandas as pd
from pathlib import Path

BASE_PATH = Path("../DATASETS/4")

post_file = BASE_PATH/'rspct_autos.tsv.gz'
subred_file = BASE_PATH/'subreddit_info.csv.gz'

In [64]:
post_df = pd.read_csv(post_file, sep = '\t')
subred_df = pd.read_csv(subred_file)

In [65]:
subred_df.set_index(['subreddit'], inplace=True)

In [66]:
df = post_df.join(subred_df, on = 'subreddit')

In [67]:
df.head(5)

,id,subreddit,title,selftext,category_1,category_2,category_3,in_data,reason_for_exclusion
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson,NaN,True,NaN
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ...",autos,ford,NaN,True,NaN
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...,autos,VW,NaN,True,NaN
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...,autos,lexus,NaN,True,NaN
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu...",autos,chevrolet,NaN,True,NaN


#### Standardizing Attributes

In [68]:
print(df.columns)

Index(['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2',
       'category_3', 'in_data', 'reason_for_exclusion'],
      dtype='object')


In [69]:
column_mapping = {
 'id': 'id',
 'subreddit': 'subreddit',
 'title': 'title',
 'selftext': 'text',
 'category_1': 'category',
 'category_2': 'subcategory',
 'category_3': None, # no data
 'in_data': None, # not needed
 'reason_for_exclusion': None # not needed
}
# define remaining columns
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]


In [70]:
df = df[columns].rename(columns = column_mapping)

In [71]:
df = df[df['category'] == 'autos']

In [72]:
df.sample(1).T

,12295
id,76ff08
subreddit,Volkswagen
title,Advice about 2006 Passat 2.0T?
text,"Hi everyone, I’m looking for a winter car and ..."
category,autos
subcategory,VW


In [73]:
import sqlite3

In [76]:
db_name = "reddit-selfposts.db"
con = sqlite3.connect(db_name)
df.to_sql("posts", con, index = False, if_exists = 'replace')
con.close()

In [77]:
con = sqlite3.connect(db_name)
df = pd.read_sql("select * from posts", con)
con.close()

## Cleaning Text

### Identify Noise with Regular Experssions

In [80]:
import re

In [81]:
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

In [89]:
def impurity(text, min_len = 0):
    '''returns the share of suspicious characters in a text'''
    len_text = len(text)
    if text == None or len_text < min_len:
        return 0
    return len(RE_SUSPICIOUS.findall(text)) / len_text

In [90]:
text = """
After viewing the [PINKIEPOOL Trailer](https://www.youtu.be/watch?v=ieHRoHUg)
it got me thinking about the best match ups.
<lb>Here's my take:<lb><lb>[](/sp)[](/ppseesyou) Deadpool<lb>[](/sp)[](/ajsly)
Captain America<lb>"""

impurity(text, 4)

0.09009009009009009

In [91]:
df['impurity'] = df['text'].apply(impurity, min_len = 10)

In [96]:
df[['text', 'impurity']].sort_values(by = 'impurity', ascending=False).head(5)

,text,impurity
19682,Looking at buying a 335i with 39k miles and 11...,0.214716
12357,I'm looking to lease an a4 premium plus automa...,0.165099
2730,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L...,0.139130
12754,Bulbs Needed:<lb><lb><lb>**194 LED BULB x8**<l...,0.132411
10726,I currently have a deposit on a 2013 335is (CP...,0.129317


In [99]:
from exp import count_words

In [101]:
count_words(df, 
            column='text', 
            preprocess=lambda doc: re.findall(r'<[\w]*>', doc)
        )

,freq
token,
<lb>,100729
<tab>,642


### Removing Noise with Regular Expressions

In [103]:
import html

In [114]:
def clean(text):
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [115]:
clean_text = clean(text)
print(clean_text)
print("Impurity: ", impurity(clean_text))

After viewing the PINKIEPOOL Trailer it got me thinking about the best match ups. Here's my take: Deadpool Captain America
Impurity:  0.0


In [117]:
df['clean_text'] = df['text'].map(clean)
df['impurity'] = df['clean_text'].apply(impurity, min_len = 20)

df[['clean_text', 'impurity']].sort_values(by = 'impurity', ascending=False).head(3)

,clean_text,impurity
14058,"Mustang 2018, 2019, or 2020? Must Haves!! 1. H...",0.030864
18934,"At the dealership, they offered an option for ...",0.026455
16505,"I am looking at four Caymans, all are in a sim...",0.024631


### Character Normalization with textacy

In [118]:
text = "The café “Saint-Raphaël” is loca-\nted on Côte dʼAzur."

In [ ]:
import textacy.pre